### 메시지 축약

In [ ]:
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    trim_messages,
)
from langchain_ollama import ChatOllama

# 샘플 메시지 설정
messages = [
    SystemMessage(content="당신은 친절한 어시스턴트입니다."),
    HumanMessage(content="안녕하세요! 나는 민혁입니다."),
    AIMessage(content="안녕하세요!"),
    HumanMessage(content="바닐라 아이스크림을 좋아해요."),
    AIMessage(content="좋네요!"),
    HumanMessage(content="2 + 2는 얼마죠?"),
    AIMessage(content="4입니다."),
    HumanMessage(content="고마워요."),
    AIMessage(content="천만에요!"),
    HumanMessage(content="즐거운가요?"),
    AIMessage(content="예!"),
]

# 축약 설정
trimmer = trim_messages(
    max_tokens=65,  # max_tokens 바꿔서 실험
    strategy="last",
    token_counter=ChatOllama(model="gemma3n:e2b"),
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# 축약 적용
trimmed = trimmer.invoke(messages)
print(trimmed)

[SystemMessage(content='당신은 친절한 어시스턴트입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요! 나는 민혁입니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='바닐라 아이스크림을 좋아해요.', additional_kwargs={}, response_metadata={}), AIMessage(content='좋네요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 + 2는 얼마죠?', additional_kwargs={}, response_metadata={}), AIMessage(content='4입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='고마워요.', additional_kwargs={}, response_metadata={}), AIMessage(content='천만에요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='즐거운가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='예!', additional_kwargs={}, response_metadata={})]


### 메시지 필터링

In [4]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    filter_messages,
)
from langchain_ollama import ChatOllama

# 샘플 메시지
messages = [
    SystemMessage(content="당신은 친절한 어시스턴트입니다.", id="1"),
    HumanMessage(content="예시 입력", id="2", name="example_user"),
    AIMessage(content="예시 출력", id="3", name="example_assistant"),
    HumanMessage(content="실제 입력", id="4", name="bob"),
    AIMessage(content="실제 출력", id="5", name="alice"),
]

# 사용자 메시지만 필터링
human_messages = filter_messages(messages, include_types="human")
print("사용자 메시지:", human_messages)

사용자 메시지: [HumanMessage(content='예시 입력', additional_kwargs={}, response_metadata={}, name='example_user', id='2'), HumanMessage(content='실제 입력', additional_kwargs={}, response_metadata={}, name='bob', id='4')]


In [5]:
# 특정 이름의 메시지 제외
excluded_names = filter_messages(
    messages, exclude_names=["example_user", "example_assistant"]
)
print("\n이름에 example이 포함되지 않은 메시지:", excluded_names)


이름에 example이 포함되지 않은 메시지: [SystemMessage(content='당신은 친절한 어시스턴트입니다.', additional_kwargs={}, response_metadata={}, id='1'), HumanMessage(content='실제 입력', additional_kwargs={}, response_metadata={}, name='bob', id='4'), AIMessage(content='실제 출력', additional_kwargs={}, response_metadata={}, name='alice', id='5')]


In [6]:
# 유형과 ID로 필터링
filtered_messages = filter_messages(
    messages, include_types=["human", "ai"], exclude_ids=["3"]
)
print("\n특정 유형과 ID로 필터링한 메시지:", filtered_messages)


특정 유형과 ID로 필터링한 메시지: [HumanMessage(content='예시 입력', additional_kwargs={}, response_metadata={}, name='example_user', id='2'), HumanMessage(content='실제 입력', additional_kwargs={}, response_metadata={}, name='bob', id='4'), AIMessage(content='실제 출력', additional_kwargs={}, response_metadata={}, name='alice', id='5')]


In [7]:
# 선언형 구성
model = ChatOllama(model="gemma3n:e2b")
filter_ = filter_messages(exclude_names=["example_user", "example_assistant"])
chain = filter_ | model

### 연속된 메시지 병합

In [8]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    merge_message_runs,
)
from langchain_ollama import ChatOllama

# 동일한 유형의 연속된 메시지 예제
messages = [
    SystemMessage(content="당신은 친절한 어시스턴트입니다."),
    SystemMessage(content="항상 농담으로 대답하세요."),
    HumanMessage(content=[{"type": "text", "text": "어떤 피자가 제일 맛있나요?"}]),
    HumanMessage(content="어떤 햄버거가 가장 맛있나요?"),
    AIMessage(content='나는 항상 너만 "고르곤졸라"'),
    AIMessage(content='너가 "버거" 싶어'),
]

# 연속된 메시지를 병합
merged = merge_message_runs(messages)
print(merged)

[SystemMessage(content='당신은 친절한 어시스턴트입니다.\n항상 농담으로 대답하세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content=[{'type': 'text', 'text': '어떤 피자가 제일 맛있나요?'}, '어떤 햄버거가 가장 맛있나요?'], additional_kwargs={}, response_metadata={}), AIMessage(content='나는 항상 너만 "고르곤졸라"\n너가 "버거" 싶어', additional_kwargs={}, response_metadata={})]


In [ ]:
# 선언형 구성
model = ChatOllama(model="gemma3n:e2b")
merger = merge_message_runs()
chain = merger | model